In [1]:
import boto3
import s3fs
import xarray as xr
import fsspec
import numpy as np
import time

In [2]:
%%time
bucket_url = 's3://noaa-nwm-retrospective-3-0-pds/CONUS/zarr/forcing/precip.zarr'

# create an instace of the S3FileSystem class from s3fs and list files
s3 = boto3.client('s3')
s3_filesystem = s3fs.S3FileSystem(anon=True)
files = s3_filesystem.ls(f'{bucket_url}') 
files

CPU times: user 291 ms, sys: 368 ms, total: 658 ms
Wall time: 7.92 s


['noaa-nwm-retrospective-3-0-pds/CONUS/zarr/forcing/precip.zarr/',
 'noaa-nwm-retrospective-3-0-pds/CONUS/zarr/forcing/precip.zarr/.zattrs',
 'noaa-nwm-retrospective-3-0-pds/CONUS/zarr/forcing/precip.zarr/.zgroup',
 'noaa-nwm-retrospective-3-0-pds/CONUS/zarr/forcing/precip.zarr/.zmetadata',
 'noaa-nwm-retrospective-3-0-pds/CONUS/zarr/forcing/precip.zarr/RAINRATE',
 'noaa-nwm-retrospective-3-0-pds/CONUS/zarr/forcing/precip.zarr/crs',
 'noaa-nwm-retrospective-3-0-pds/CONUS/zarr/forcing/precip.zarr/time',
 'noaa-nwm-retrospective-3-0-pds/CONUS/zarr/forcing/precip.zarr/x',
 'noaa-nwm-retrospective-3-0-pds/CONUS/zarr/forcing/precip.zarr/y']

In [3]:
%%time
conus_bucket_url = 's3://noaa-nwm-retrospective-3-0-pds/CONUS/zarr/forcing/precip.zarr'

ds = xr.open_zarr(
        fsspec.get_mapper(
            conus_bucket_url,
            anon=True
        ),
        consolidated=True
)
ds

ValueError: unrecognized engine zarr must be one of: ['netcdf4', 'scipy', 'rasterio', 'store']

In [5]:
ds_input_coords = xr.open_dataset("my_dataset.nc")
ds_input_coords

<xarray.Dataset> Size: 178kB
Dimensions:     (point: 419, coordinate: 2)
Coordinates:
  * point       (point) int64 3kB 0 1 2 3 4 5 6 ... 412 413 414 415 416 417 418
  * coordinate  (coordinate) <U3 24B 'lon' 'lat'
Data variables: (12/26)
    10011500    (point, coordinate) int64 7kB ...
    10105900    (point, coordinate) int64 7kB ...
    10109000    (point, coordinate) int64 7kB ...
    10126000    (point, coordinate) int64 7kB ...
    10129900    (point, coordinate) float64 7kB ...
    10131000    (point, coordinate) int64 7kB ...
    ...          ...
    10154200    (point, coordinate) float64 7kB ...
    10155000    (point, coordinate) int64 7kB ...
    10155200    (point, coordinate) int64 7kB ...
    10157500    (point, coordinate) float64 7kB ...
    10164500    (point, coordinate) float64 7kB ...
    10172952    (point, coordinate) float64 7kB ...

In [13]:
%%time
data = ds_input_coords
all_datasets = []


for dataset in data:
        start_time = time.time()
        print(f'{dataset} started')
        temp_lat = data[dataset].sel(coordinate='lat')
        temp_lat = temp_lat.where(np.logical_not(np.isnan(temp_lat)), drop=True)
        temp_lon = data[dataset].sel(coordinate='lon')
        temp_lon = temp_lon.where(np.logical_not(np.isnan(temp_lon)), drop=True)
        precip = ds.isel(x=temp_lat.astype(int), y=temp_lon.astype(int)) #.squeeze().RAINRATE
        precip_1 = precip.RAINRATE.resample(time='D').sum() * 24 * 3600
        precip_2 = precip_1.mean(dim='point')
        precip_2['station'] = dataset
        all_datasets.append(precip_2)
        precip_2.to_zarr(f'{dataset}.zarr', mode='w')
        print(f'{dataset} ended')
        print(f'time {int((time.time() - start_time) // 60)} minutes, {int((time.time() - start_time) % 60)} seconds ')


combined_data = xr.concat(all_datasets, dim="station")


10011500 started
10011500 ended
time 3 minutes, 42 seconds 
10105900 started
10105900 ended
time 3 minutes, 5 seconds 
10109000 started
10109000 ended
time 3 minutes, 6 seconds 
10126000 started
10126000 ended
time 3 minutes, 5 seconds 
10129900 started
10129900 ended
time 3 minutes, 7 seconds 
10131000 started
10131000 ended
time 3 minutes, 5 seconds 
10133650 started
10133650 ended
time 3 minutes, 5 seconds 
10133800 started
10133800 ended
time 3 minutes, 5 seconds 
10133980 started
10133980 ended
time 3 minutes, 4 seconds 
10134500 started
10134500 ended
time 3 minutes, 4 seconds 
10136500 started
10136500 ended
time 3 minutes, 5 seconds 
10137500 started
10137500 ended
time 2 minutes, 59 seconds 
10140100 started
10140100 ended
time 2 minutes, 38 seconds 
10140700 started
10140700 ended
time 2 minutes, 59 seconds 
10141000 started
10141000 ended
time 3 minutes, 7 seconds 
10145400 started
10145400 ended
time 3 minutes, 8 seconds 
10146400 started
10146400 ended
time 3 minutes, 14 s

In [14]:
combined_data

<xarray.DataArray 'RAINRATE' (station: 26, time: 16071)> Size: 2MB
dask.array<concatenate, shape=(26, 16071), dtype=float32, chunksize=(1, 1), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 129kB 1979-02-01 1979-02-02 ... 2023-01-31
  * station  (station) <U8 832B '10011500' '10105900' ... '10164500' '10172952'

In [23]:
%%time
combined_data.to_zarr(f'final_rain.zarr', mode='w')

In [16]:
%%time
df_combined = combined_data.to_dataframe()

CPU times: user 27min 7s, sys: 5min 55s, total: 33min 3s
Wall time: 18min 10s


In [17]:
%%time
df_combined_modified = df_combined.reset_index()

CPU times: user 11.7 ms, sys: 210 µs, total: 11.9 ms
Wall time: 28 ms


In [22]:
df_combined_modified.dtypes
df_combined_modified.to_csv('final_rain.csv')

In [21]:
df_combined_modified

,station,time,RAINRATE
0,10011500,1979-02-01,109.788467
1,10011500,1979-02-02,6.566367
2,10011500,1979-02-03,0.965529
3,10011500,1979-02-04,0.000000
4,10011500,1979-02-05,0.000000
...,...,...,...
417841,10172952,2023-01-27,0.000000
417842,10172952,2023-01-28,0.000000
417843,10172952,2023-01-29,0.000000
417844,10172952,2023-01-30,12.361973
